# 1. Import

In [5]:
%%capture
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import math 
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import re
import pickle
import boto
from copy import deepcopy
import json
import os
import jinja2
import random

from mturk import MTurk

from boto.mturk.qualification import PercentAssignmentsApprovedRequirement, Qualifications, Requirement, LocaleRequirement

from keysTkingdom import mturk_ai2
# from keysTkingdom import aws_tokes
# from keysTkingdom import mturk_aristo

# 2. Submitting HITs- metric collection

In [3]:
from jinja2 import Environment, FileSystemLoader
import random

methods = ['nn', 'pix2pix', 'ours']

def write_task_page(page_html):
    html_dir = './html_renders'
    html_out_file = os.path.join(html_dir, 'img_rate.html')
    if not os.path.exists(html_dir):
        os.makedirs(html_dir)
    with open(html_out_file, 'w') as f:
        f.write(page_html)

def generate_task_page(s3_base_path, img, template_file='stage_3a.html'):
    env = Environment(loader=FileSystemLoader('hit_templates'))
    template = env.get_template(template_file)
#     image_url = os.path.join(s3_base_path, 'gt', img)
    permute_methods = random.sample(methods, len(methods))
    page_html = template.render(s3_uri_base=s3_base_path, image_url=img, method1=permute_methods[0],  method2=permute_methods[1],  method3=permute_methods[2])
    page_html = page_html
    return page_html

def prepare_hit(s3_base_path, img_uri, static_parameters, task_generator=generate_task_page):
    question_html = task_generator(s3_base_path, img_uri)
    return build_hit_params(question_html, static_parameters)


def build_hit_params(qhtml, static_params):
    """
    Dynamically builds some HIT params that will change based on the book/url
    :param url: formatted url of page image on s3
    :param static_params: Universal HIT params (set by user in notebook).
    :return: complete HIT parameters.
    """
    import copy
    import boto

    def build_qualifications(locales=None):
        """
        Creates a single qualification that workers have a > 95% acceptance rate.
        :return: boto qualification obj.
        """
        qualifications = Qualifications()
        requirements = [PercentAssignmentsApprovedRequirement(comparator="GreaterThan", integer_value="95")]
        if locales:
            loc_req = LocaleRequirement(
                    comparator='In',
                    locale=locales)
            requirements.append(loc_req)
        _ = [qualifications.add(req) for req in requirements]
        return qualifications
    if 'locales' not in static_params:
        static_params['locales'] = None
    hit_params = copy.deepcopy(static_params)
    hit_params['qualifications'] = build_qualifications(static_params['locales'])
    hit_params['reward'] = boto.mturk.price.Price(hit_params['amount'])
    hit_params['html'] = qhtml
    return hit_params

In [6]:
turk_account = mturk_ai2
rw_host='mechanicalturk.amazonaws.com'
# amt_con = MTurk(turk_account.access_key, turk_account.access_secret_key, host=rw_host)
amt_con = MTurk(turk_account.access_key, turk_account.access_secret_key)
amt_con.get_account_balance()

$10,000.00

In [7]:
static_params = {
    'title': "Select the image that's most similar to a given source image",
    'description': "You will be shown a source image and asked to select the most closely matching image from 3 options",
    'keywords': ['images'],
    'frame_height': 1000,
    'amount': 0.02,
    'duration': 3600 * 1,
    'lifetime': 3600 * 24 * 2,
    'max_assignments': 1,
    'locales': ['US', 'CA', 'AU', 'NZ', 'GB']
}

In [8]:
s3_base_path = 'https://s3-us-west-2.amazonaws.com/ai2-vision-animation-gan/annotation_data/user_study/'

In [2]:
gt_names = !ls ./data/user_study/groundtruth
nn_names = !ls ./data/user_study/nn
ours_names = !ls ./data/user_study/ours
pix2pix_names = !ls ./data/user_study/pix2pix/

In [9]:
build_hit_group = [prepare_hit(s3_base_path, img, static_params) for img in tqdm(gt_names[:10])]

write_task_page(random.choice(build_hit_group)['html'])

100%|██████████| 10/10 [00:00<00:00, 156.70it/s]


In [106]:
# hit_group = [amt_con.create_html_hit(single_hit) for single_hit in tqdm(build_hit_group)]

100%|██████████| 4357/4357 [16:28<00:00,  4.41it/s]


# 3. Retrieve results

In [107]:
# %%time
# all_hits = amt_con.get_reviewable_hits(detailed=False)

In [163]:
%%time
all_hits = amt_con.get_all_hits()

start_date = (2018, 1, 23)
end_date = (2018, 1, 27)
recent_hits = filter_hits_by_date(all_hits, start_date, end_date)
recent_hits = filter_hits_by_status(recent_hits)

CPU times: user 2.08 s, sys: 60.5 ms, total: 2.14 s
Wall time: 32.2 s


In [165]:
%%time
results = get_assignments(amt_con.connection, recent_hits)

CPU times: user 11.3 s, sys: 484 ms, total: 11.8 s
Wall time: 7min 52s


In [129]:
def create_result(assmt):
    result = json.loads(assmt.answers[0][0].fields[0])
    result['h_id'] = assmt.HITId
    result['a_id'] = assmt.AssignmentId
    result['worker_id'] = assmt.WorkerId
    return result

In [176]:
proc_results = []
for hid, ar in results.items():
    if ar:
        proc_results.append(create_result(ar[0]))

final_results = {v['image_url'].split('/')[-1]: v['selected_image'] for v in proc_results}
res_series = pd.Series(final_results)

# 4. Accepting and deleting HITs

Uncomment only when ready to accept or delete hits

reject assignments carefully

In [166]:
assignments =[]
for assignment_triple in list(results.values()):
    assignments.extend(assignment_triple)

len(assignments)

4355

In [170]:
ta = assignments[-1]

u'Approved'

In [173]:
assignments_to_accept = [asgmt for asgmt in assignments if asgmt.AssignmentStatus != 'Approved']

In [174]:
len(assignments_to_accept)

23

In [175]:
e_count = 0
for assignment in tqdm(assignments_to_accept):
    try:
        amt_con.approve_assignment(assignment)
    except boto.mturk.connection.MTurkRequestError as e:
        e_count += 1

100%|██████████| 23/23 [00:04<00:00,  5.69it/s]


In [162]:
e_count

0

In [ ]:
print(e_count)

In [ ]:
# _ = [amt_con.disable_hit(hit) for hit in hits_today]

In [ ]:
# number_rejected_assignments, number_rejected_workers = amt_util.reject_assignments(mturk, workers_to_ban, combined_consensus_with_workerid_df)
# print 'rejecting ' + str(number_rejected_assignments) + ' assignments' + ' from ' + str(number_rejected_workers) + ' workers'

In [41]:
amt_con.get_account_balance()

$10,000.00

In [42]:
amt_con.delete_all_hits()

In [178]:
# _ = [amt_con.disable_hit(hit) for hit in tqdm(all_hits)]

In [43]:
e_count = 0
for hitid in tqdm(all_hits):
    try:
        amt_con.disable_hit(hitid[0])
    except boto.mturk.connection.MTurkRequestError as e:
        e_count += 1

100%|██████████| 64326/64326 [2:07:16<00:00,  6.91it/s]  
